In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import csv
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import gc
from random import randint

from typing import List, Dict
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset

In [3]:
classes = []
with open("/content/drive/MyDrive/Colab_Notebooks/mini/category.csv", 'r') as csvfile:
  # creating a csv reader object
  csvreader = csv.reader(csvfile)
  _ = next(csvreader)
  for i in csvreader:
    classes.append(i[1])

In [4]:
with open("/content/drive/MyDrive/Colab_Notebooks/mini/test_filename.txt", 'r') as f:
  filenames = f.read()
  filenames = filenames.split(" ")

print(len(filenames))
filenames.remove("")
print(len(filenames))

3900
3899


In [5]:
file_idx = []
for i in filenames:
  tmp = i.split(".")
  file_idx.append(int(tmp[0]))

file_idx.sort()

for i in range(len(file_idx)):
  filenames[i] = str(file_idx[i]) + "." + tmp[1]

In [6]:
!unzip '/content/drive/MyDrive/Colab_Notebooks/mini/test_crop.zip' -d '/content'

Archive:  /content/drive/MyDrive/Colab_Notebooks/mini/test_crop.zip
  inflating: /content/test_crop/0.jpg  
  inflating: /content/test_crop/10.jpg  
  inflating: /content/test_crop/100.jpg  
  inflating: /content/test_crop/1000.jpg  
  inflating: /content/test_crop/1001.jpg  
  inflating: /content/test_crop/1002.jpg  
  inflating: /content/test_crop/1003.jpg  
  inflating: /content/test_crop/1004.jpg  
  inflating: /content/test_crop/1005.jpg  
  inflating: /content/test_crop/1006.jpg  
  inflating: /content/test_crop/1007.jpg  
  inflating: /content/test_crop/1008.jpg  
  inflating: /content/test_crop/1009.jpg  
  inflating: /content/test_crop/1010.jpg  
  inflating: /content/test_crop/1011.jpg  
  inflating: /content/test_crop/1012.jpg  
  inflating: /content/test_crop/1013.jpg  
  inflating: /content/test_crop/1014.jpg  
  inflating: /content/test_crop/1015.jpg  
  inflating: /content/test_crop/1016.jpg  
  inflating: /content/test_crop/1017.jpg  
  inflating: /content/test_crop/101

In [7]:
testset = []
img_path = "/content/test_crop/"

for i in range(len(filenames)):
  img = cv2.imread(img_path + filenames[i])
  img = cv2.resize(img, (224, 224))
  img = img.astype("float32") / 255.0
  testset.append(torch.from_numpy(img))
  print(filenames[i])

0.jpg
2.jpg
3.jpg
4.jpg
7.jpg
9.jpg
10.jpg
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
19.jpg
20.jpg
22.jpg
23.jpg
24.jpg
25.jpg
26.jpg
30.jpg
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.jpg
38.jpg
39.jpg
40.jpg
41.jpg
42.jpg
43.jpg
45.jpg
46.jpg
48.jpg
49.jpg
50.jpg
51.jpg
52.jpg
53.jpg
54.jpg
55.jpg
56.jpg
57.jpg
58.jpg
59.jpg
60.jpg
61.jpg
62.jpg
64.jpg
66.jpg
70.jpg
71.jpg
72.jpg
73.jpg
77.jpg
78.jpg
79.jpg
80.jpg
81.jpg
82.jpg
83.jpg
84.jpg
85.jpg
86.jpg
87.jpg
88.jpg
89.jpg
90.jpg
91.jpg
92.jpg
93.jpg
94.jpg
96.jpg
99.jpg
100.jpg
102.jpg
103.jpg
104.jpg
106.jpg
108.jpg
109.jpg
110.jpg
113.jpg
115.jpg
116.jpg
117.jpg
118.jpg
120.jpg
121.jpg
122.jpg
124.jpg
125.jpg
126.jpg
127.jpg
128.jpg
129.jpg
130.jpg
131.jpg
134.jpg
135.jpg
136.jpg
137.jpg
138.jpg
139.jpg
140.jpg
141.jpg
142.jpg
144.jpg
145.jpg
146.jpg
147.jpg
149.jpg
150.jpg
152.jpg
154.jpg
157.jpg
159.jpg
161.jpg
164.jpg
170.jpg
172.jpg
174.jpg
176.jpg
177.jpg
178.jpg
179.jpg
182.jpg
184.jpg
185.jpg
186.jpg
187.jpg
188.jpg
189

In [8]:
print(f'Can I can use GPU now? -- {torch.cuda.is_available()}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Can I can use GPU now? -- True


In [10]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 100)
transform = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

stat_dict = torch.load("/content/drive/MyDrive/Colab_Notebooks/mini/model_0.pth")
model.load_state_dict(stat_dict)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
pred = []
model = model.to(device)

count = 0
for p in range(len(testset)):
  i = testset[p]
  i = i.view(1, 224, 224, 3)
  i = i.permute(0, 3, 1, 2)
  i = i.to(device)
  with torch.no_grad():
    output = model(i)
    pred.append(output.data.max(1, keepdim=True)[1])
    print(pred[-1])

tensor([[43]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([[38]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[38]], device='cuda:0')
tensor([[38]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[38]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[94]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[38]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[38]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[43]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([

In [12]:
print(len(pred))
print(len(filenames))
prediction = []
for p in range(4977):
  name = str(p) + ".jpg"
  if name not in filenames:
    prediction.append([p, classes[randint(0,99)]])
  else:
    i = filenames.index(name)
    prediction.append([p, classes[pred[i].item()]])

with open("/content/drive/MyDrive/Colab_Notebooks/mini/pred.csv", 'w', newline='') as f:
  writer = csv.writer(f)
  writer.writerow(["Id", "Category"])
  writer.writerows(prediction)
  f.close()

3899
3899
